<a href="https://colab.research.google.com/github/hjesse92/style_transfer_w266/blob/main/Few_Shot_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few Show Learning

## Setup

In [1]:
!pip install -q transformers datasets sentencepiece rouge_score accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.5 MB/s eta 0:00

In [2]:
#Am I running a GPU and what type is it?
!nvidia-smi

Sat Mar 11 04:37:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('Number of GPU(s) available:', torch.cuda.device_count())
    print('GPU device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available')
    device = torch.device("cpu")

Number of GPU(s) available: 1
GPU device name: Tesla T4


In [4]:
from logging import warning
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW, Seq2SeqTrainingArguments, Seq2SeqTrainer


from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import re
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pprint

import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!cd drive/MyDrive

In [6]:
train_file = 'drive/MyDrive/data/original-train.tsv'
dev_file = 'drive/MyDrive/data/original-dev.tsv'
test_file = 'drive/MyDrive/data/original-test.tsv'
df_train = pd.read_csv(train_file, sep='\t')
df_dev = pd.read_csv(dev_file, sep='\t')
df_test = pd.read_csv(test_file, sep='\t')

In [7]:
print(f'''mean length of offensive text: {df_train['offensive-text'].map(len).mean()}''')
print(f'''min length of offensive text: {df_train['offensive-text'].map(len).min()}''')
print(f'''max length of offensive text: {df_train['offensive-text'].map(len).max()}''')
print(f'''mean length of neutralized text: {df_train['style-transferred-text'].map(len).mean()}''')
print(f'''min length of neutralized text: {df_train['style-transferred-text'].map(len).min()}''')
print(f'''max length of neutralized text: {df_train['style-transferred-text'].map(len).max()}''')

mean length of offensive text: 69.85353535353535
min length of offensive text: 9
max length of offensive text: 238
mean length of neutralized text: 60.48800505050505
min length of neutralized text: 1
max length of neutralized text: 174


# Trial with Flan T5

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

t5tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto", torch_dtype=torch.float16)

In [ ]:
#@title 3-Shot Learning
df_fewshot = df_test.copy()
n_shots = 3

for row in range(len(df_fewshot)):
    #For each item in the test set, we use 3 examples from the training set to as n-shots
    sources_targets = df_train.sample(n_shots, replace=False)

    for i in range(n_shots):
        df_fewshot.loc[row, f'shot{i+1}_source'] = sources_targets.iloc[i,0]
        df_fewshot.loc[row, f'shot{i+1}_target'] = sources_targets.iloc[i,1]


df_fewshot = df_fewshot.rename(columns={'offensive-text':'source', 'style-transferred-text':'target'})

In [ ]:
df_fewshot['prompt'] = df_fewshot.apply(lambda x: 
                 'Rewrite the toxic text in non-toxic style. \n\n'
                 'Toxic text: ' + x['shot1_source'] + '\n' + 'Non-toxic text: ' + x['shot1_target'] + '\n\n' + \
                 'Toxic text: ' + x['shot2_source'] + '\n' + 'Non-toxic text: ' + x['shot2_target'] + '\n\n' + \
                 'Toxic text: ' + x['shot3_source'] + '\n' + 'Non-toxic text: ' + x['shot3_target'] + '\n\n' + \
                 'Toxic text: ' + x['source'] + '\n' + 'Non-toxic text: ', axis=1)

In [ ]:
print(df_fewshot.prompt[0])

Rewrite the toxic text in non-toxic style. 

Toxic text: Has been a US military puppet from jump
Non-toxic text: Has been a tool for the US military from the beginning

Toxic text: It's not though you fucking idiot
Non-toxic text: It's not though

Toxic text: Maybe you shouldn't turn your new country into the shithole you just left.
Non-toxic text: You should be careful not to bring your old problems from your old country to your new country.

Toxic text: So maybe you should be more retarded.
Non-toxic text: 


In [ ]:
for i in range(len(df_fewshot)):
    if i%10 == 0:
        print(f'Working on number: {i}')
    t5_inputs = t5tokenizer([df_fewshot.prompt[i]], return_tensors='pt').input_ids.to("cuda")
    t5_summary_ids = t5model.generate(t5_inputs, 
                                    num_beams=5,
                                    no_repeat_ngram_size=1,
                                    num_return_sequences=1,
                                    top_p = 0.92,
                                    top_k = 100,
                                    min_length = 1,
                                    max_length = 200,
                                    do_sample=True,
                                    )
    output = t5tokenizer.batch_decode(t5_summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    df_fewshot.loc[i,'fewshot_output'] = output

Working on number: 0
Working on number: 10
Working on number: 20
Working on number: 30
Working on number: 40
Working on number: 50
Working on number: 60
Working on number: 70
Working on number: 80
Working on number: 90
Working on number: 100
Working on number: 110
Working on number: 120
Working on number: 130
Working on number: 140
Working on number: 150
Working on number: 160
Working on number: 170
Working on number: 180
Working on number: 190


In [ ]:
df_fewshot.loc[:,['source','target','prompt','fewshot_output']]

,source,target,prompt,fewshot_output
0,So maybe you should be more retarded.,So maybe you should be more backward,Rewrite the toxic text in non-toxic style. \n\...,You should be more retarded.
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,Rewrite the toxic text in non-toxic style. \n\...,THERE IS A MEGATHREAD FOR VACCINE OR COVIDE RE...
2,the fuck.... you are on drugs,uh..... you are not being realistic,Rewrite the toxic text in non-toxic style. \n\...,the fuck.... you are on drugs
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,Rewrite the toxic text in non-toxic style. \n\...,Tablemanners are no rocket science. Treating o...
4,Youre soft as baby shit,Youre really soft,Rewrite the toxic text in non-toxic style. \n\...,You're soft as baby shit.
...,...,...,...,...
194,"NTA, Jes-us your brother is dumb. How long did...","NTA, your brother is not thinking straight. Ho...",Rewrite the toxic text in non-toxic style. \n\...,Jes-us your brother is dumb. How long did he t...
195,Formula one…nascar….shit it exactly the same e...,Formula one…nascar….it exactly the same except...,Rewrite the toxic text in non-toxic style. \n\...,Formula one...nascar....shit it exactly the sa...
196,"BRB, gonna go call the mods pathetic egotistic...","BRB, gonna go call the mods.",Rewrite the toxic text in non-toxic style. \n\...,"BRB, gonna go call the mods."
197,CUCKOLD Carlson is a problem,Carlson is the problem,Rewrite the toxic text in non-toxic style. \n\...,Carlson is a problem.


In [ ]:
df_fewshot.to_csv('test_output.csv',sep='\t',index=False)

## Evaluation

In [ ]:
import evaluate

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
# bleurt = evaluate.load('bleurt')

In [ ]:
#@title Baseline Score on the source and target
print(rouge.compute(predictions=df_fewshot.source,
              references=df_fewshot.target))
print(bleu.compute(predictions=df_fewshot.source,
              references=df_fewshot.target))

# If my predictions did nothing but repeat the same toxic text, I'd get these scores

{'rouge1': 0.6887918451280337, 'rouge2': 0.5605071385408666, 'rougeL': 0.6829326508983988, 'rougeLsum': 0.6845511504130748}
{'bleu': 0.5391232310503405, 'precisions': [0.6839945280437757, 0.5702752293577982, 0.49604117181314333, 0.436613665663945], 'brevity_penalty': 1.0, 'length_ratio': 1.1498230436492332, 'translation_length': 2924, 'reference_length': 2543}


In [ ]:
#@title Score after few shot learning
print(rouge.compute(predictions=df_fewshot.fewshot_output,
              references=df_fewshot.target))
print(bleu.compute(predictions=df_fewshot.fewshot_output,
              references=df_fewshot.target))

{'rouge1': 0.6425955601886706, 'rouge2': 0.496170261974479, 'rougeL': 0.6344722227896112, 'rougeLsum': 0.6359447933623146}
{'bleu': 0.45508569897921963, 'precisions': [0.6467203682393556, 0.4962624584717608, 0.4051607062019013, 0.32985074626865674], 'brevity_penalty': 1.0, 'length_ratio': 1.0251671254423909, 'translation_length': 2607, 'reference_length': 2543}


# Tune the model with training set, then do few-shot learning again

In [32]:
from datasets import load_metric, load_dataset

In [21]:
class ToxicData(Dataset):
    def __init__(self, data, tokenizer):
        super(Dataset, self).__init__()
        self.source_texts = 'Toxic text: ' + data['offensive-text']
        self.target_texts = 'Neutralized text: ' + data['style-transferred-text']
        self.tokenizer=tokenizer

    def __len__(self):
        return len(self.target_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        source_tokens = self.tokenizer.encode(source_text, padding='max_length',truncation=True)
        source_tensor = torch.tensor(source_tokens, dtype=torch.long)

        target_tokens = self.tokenizer.encode(target_text, padding='max_length',truncation=True)
        target_tensor = torch.tensor(target_tokens, dtype=torch.long)

        if torch.cuda.is_available():
            source_tensor = source_tensor.cuda()
            target_tensor = target_tensor.cuda()

        return source_tensor, target_tensor

In [42]:
dataset = load_dataset(path="/content/drive/MyDrive/data/")

  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
dataset

DatasetDict({
    train: Dataset({
        features: ['offensive-text', 'style-transferred-text'],
        num_rows: 1584
    })
    test: Dataset({
        features: ['offensive-text', 'style-transferred-text'],
        num_rows: 199
    })
    validation: Dataset({
        features: ['offensive-text', 'style-transferred-text'],
        num_rows: 198
    })
})

In [55]:
# prefix = "summarize: "
# max_input_length = 512
# max_target_length = 64

# def clean_text(text):
#   sentences = nltk.sent_tokenize(text.strip())
#   sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
#   sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
#                                  if len(sent) > 0 and
#                                  sent[-1] in string.punctuation]
#   text_cleaned = "\n".join(sentences_cleaned_no_titles)
#   return text_cleaned

def preprocess_data(examples):
#   texts_cleaned = [clean_text(text) for text in examples["text"]]
  source_inputs = ['Toxic Text: ' + text for text in examples['offensive-text']]
  model_inputs = t5tokenizer(source_inputs, truncation=True)
  
  target_inputs = ['Non-Toxic Text: ' + text for text in examples['style-transferred-text']]
  target_tokens = t5tokenizer(target_inputs, truncation=True)

  # Setup the tokenizer for targets
#   with t5tokenizer.as_target_tokenizer():
#     labels = t5tokenizer(examples["title"], max_length=max_target_length, 
#                        truncation=True)

#   model_inputs['input_ids'] = source_tokens
  model_inputs["labels"] = target_tokens
  return model_inputs

In [57]:
tokenized_datasets = dataset.map(preprocess_data)

In [47]:
batch_size = 8
model_name = "google/flan-t5-base-tuned-toxicity"
model_dir = f"drive/MyDrive/Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard",
    remove_unused_columns=False
)

PyTorch: setting up devices


In [48]:
data_collator = DataCollatorForSeq2Seq(t5tokenizer)
metric = load_metric("rouge")

In [49]:
import nltk

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = t5tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, t5tokenizer.pad_token_id)
    decoded_labels = t5tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != t5tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [58]:
model_ckpt = "google/flan-t5-base"
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_ckpt, return_dict=True)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=t5tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
 

In [59]:
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-03-11 05:23:01.188155: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 05:23:01.188264: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 05:23:01.188276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" an

In [60]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
 

ValueError: ignored

In [22]:
# t5model.to(device)

train_data = ToxicData(df_train, t5tokenizer)
dev_data = ToxicData(df_dev, t5tokenizer)

In [23]:
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=2, shuffle=True)

In [28]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"

In [29]:
optimizer = AdamW(t5model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
num_epochs = 10

for epoch in range(num_epochs):
    # Training step
    t5model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        source_batch, target_batch = batch
        
        # Encode the source text with Flaubert and feed it into the T5 model
        outputs = t5model(input_ids=source_batch, labels=target_batch)
        loss = outputs.loss
        
        # Backpropagate the loss and update the weights
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    t5model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dev_loader:
            source_batch, target_batch = batch
            outputs = t5model(input_ids=source_batch, labels=target_batch)
            loss = outputs.loss
            total_loss += loss.item() * len(source_batch)

    # Print the loss for each epoch
    train_loss = loss.item()
    eval_loss = total_loss / len(eval_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Eval Loss: {eval_loss:.4f}")

NameError: ignored